<a href="https://colab.research.google.com/github/AllanPin/minha_pipeline_etl/blob/main/minha_pipe_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Contexto da minha pipeline:**

- eu já atingi meu limite de tokens da OpenAI, então não posso me utilizar da API deles para poder enviar aquelas mensagens super legais e personalizadas;

- nem por isso eu vou deixar de trazer criatividade. Resolvi fazer uma busca por vários modelos de ETL e montei um pipeline do meu jeito;

- apesar de sair do contexto bancário, que era a proposta inicial do grande Venilton, porém a essência fica: o aprendizado;

- ainda vamos ter uma api rest, com enfoque direto e indireto em ciência de dados.

To do:

>- criar minha api rest, para que possamos consumir seu endpoint como fonte de dados na nossa camada de extração;

>- fazer a transformação/enriquecimento dos dados;

>- enviar os dados de volta para a nossa api

In [ ]:
# criar a api e salvar no swagger, será que dá certo?

# **Extraindo** os dados de treinamento

Keras é uma API feita com Python e roda com base no TensorFlow, o modelo de deep learning feito pelo time do Google Brain. Ao lado do PyTorch, é um dos modelos mais utilizados nesse ramo.

Keras conta com alguns datasets que podemos carregar como nossa fonte de dados primária. Dentre esses datasets, o que escolhemos foi o Fashion MNIST, uma base com 10 mil imagens para treinarmos o modelo e mais 60 mil para testarmos o modelo.

Esse Fashion MNIST, como o nome sugere, possui imagens cujos rótulos são categorizados da seguinte maneira:

>| Rótulo | Descrição | Tradução
 | -------| ----------| ----------|
 | 0 | T-shirt/top| Camiseta |
 | 1 | Trouser | Calças |
 | 2 | Pullover | Pulôver |
 | 3 | Dress | Vestido |
 | 4 | Coat | Casaco |
 | 5 | Sandal | Sandália |
 | 6 | Shirt | Camisa |
 | 7 | Sneaker | Tênis |
 | 8 | Bag | Bolsa |
 | 9 | Ankle boot | Bota de Salto |

 Ps.: markdown é genial!

In [3]:
import numpy as np
import tensorflow.keras as keras

(xtrain, ytrain), (xtest, ytest) = keras.datasets.fashion_mnist.load_data()

xtrain = xtrain.astype('float32') / 255
xtest = xtest.astype('float32') / 255

xtrain = np.reshape(xtrain, (xtrain.shape[0], 28, 28, 1))
xtest = np.reshape(xtest, (xtest.shape[0], 28, 28, 1))

print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(xtest.shape)

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000, 28, 28, 1)


# **Carregando** e **Transformando** os dados em um banco de dados sql e em seguida um DataFrame do pandas

Nessa etapa, iremos armazenar os dados em um banco de dados SQL. Vamos utilizar o pandas também, para visualização das tabelas.

In [6]:
import sqlite3
import pandas as pd

In [7]:
# faz a conexão com o sqlite3, através de database chamado fashion_mnist
conn = sqlite3.connect('fashion_mnist.db')

# criamos a tabela para receber as imagens
# blob é literalmente uma bolha de dados. Isso faz sentido para as imagens
# pois uma imagem nada mais é do que uma matriz de pixels. E, para contextualizar
# melhor, as imagens de fashion_mnist são de gradientes de cinza, i.e., cada
# pixel terá um valor individual entre 0 e 255, sendo 0 -> preto e 255 -> branco.
# É importante destacar também que as imagens de nossa base são de resolução
# 28 x 28, totalizando 784 pixels por imagem
conn.execute('''CREATE TABLE IF NOT EXISTS images
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
             image BLOB NOT NULL,
             label INTEGER NOT NULL);''')

# Esse loop é o responsável por "popular" a nossa tabela com as imagens e
# seus respectivos rótulos. Os pontos de interrogação representam os valores que
# serão inseridos nas colunas "image" e "label" da tabela "images"
for i in range(xtrain.shape[0]):
    conn.execute('INSERT INTO images (image, label) VALUES (?, ?)',
                [sqlite3.Binary(xtrain[i]), ytrain[i]])

# commitamos as mudanças feitas. Tipo no git (se eu entendesse git...)
conn.commit()

#O mesmo que o loop anterior, porém com as imagens de teste
for i in range(xtest.shape[0]):
    conn.execute('INSERT INTO images (image, label) VALUES (?, ?)',
                [sqlite3.Binary(xtest[i]), ytest[i]])

# commitamos novamente as inserções feitas
conn.commit()

conn.close()

Exemplo de como acessar os dados armazenados:

In [9]:
conn = sqlite3.connect('fashion_mnist.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM images')
rows = cursor.fetchall()

# passamos todas as imagens para um DataFrame do pandas
# desse modo a gente consegue trabalhar com os dados mas sem o "engessamento"
# da linguagem sql (perdão pela ofensa, dba's do mundo todo)
data = pd.read_sql_query('SELECT * FROM images', conn)

# **Conclusão**

Esse pipeline ETL é um exemplo muito rico e que pode ser o fundamento para a elaboração e treinamento de um modelo de Deep Learning que possa "visualizar" imagens e nos dizer se há alguma peça de vestuário. Sem falar que poderíamos facilmente alterarmos a fonte de dados na seção de extração e mesmo assim conseguiríamos manter o perfil do restante da pipeline quase que intacto (ainda precisaríamos modificar alguns detalhes, como por exemplo o nome das variáveis para que o código mantenha um certo contexto).